<a href="https://colab.research.google.com/github/pinei/ciencia-dados-python/blob/main/imersao-dados/5_Valida%C3%A7%C3%A3o_de_Modelo_e_Overfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Amostra de dados do ENEM (dados originais disponiveis em http://portal.inep.gov.br/microdados)
fonte = "https://raw.githubusercontent.com/alura-cursos/imersao-dados-2-2020/master/MICRODADOS_ENEM_2019_SAMPLE_43278.csv"
dados_df = pd.read_csv(fonte)  # type: <class 'pandas.core.frame.DataFrame'>

# Ciências da Natureza, Ciências Humanas, Matemática, Linguagens e Codigos, Redação
provas = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO"]

# Soma das Notas é dada pela Soma Horizontal das variáveis que representam as Notas (axis=1)
# Precisa de uma Nota válida para ser considerada, se tornando NaN se todas a Notas foram NaN (min_count=1)
dados_df["NU_NOTA_TOTAL"] = dados_df[provas].sum(axis=1, min_count=1) # type: pandas.core.series.Series

In [ ]:
dados_notas_df = dados_df[provas].dropna() # Retira os valores NaN

# Variáveis de entrada e saída para o Machine Learning
provas_entrada = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
provas_saida = "NU_NOTA_MT"

notas_entrada = dados_notas_df[provas_entrada]
notas_saida = dados_notas_df[provas_saida]

In [ ]:
# Dados de Treino e Teste do modelo de Machine Learning
from sklearn.model_selection import train_test_split

x = notas_entrada
y = notas_saida

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25)

In [ ]:
# Modelo Support Vector Regression (SVR)
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

# Rodando o modelo com o fator de aleatoriedade implicito
modelo = LinearSVR()
modelo.fit(x_treino, y_treino)
predicoes_notas_mt = modelo.predict(x_teste)

# O "mean squared error" em diferentes rodadas está variando entre 6000 a 23000.
mean_squared_error(y_teste, predicoes_notas_mt)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


6136.983750398868

In [ ]:
# Precisamos reduzir tentando outros modelos a aleatoriedade do resultado anterior.

# Modelo Decision Tree
from sklearn.tree import DecisionTreeRegressor

# A separação dos dados de treino e teste já impõe um fator de aleatoriedade no resultado do modelo.
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25)

# O modelo Decision Tree tem um fator muito baixo de aleatoriedade
modelo_arvore = DecisionTreeRegressor(max_depth = 3)
modelo_arvore.fit(x_treino, y_treino)
predicoes_notas_mt_arvore = modelo.predict(x_teste)

# O "mean squared error" em diferentes rodadas está variando entre 6055 e 6210,
# praticamente em função dos dados de treino e teste que estão variando.
mean_squared_error(y_teste, predicoes_notas_mt_arvore)

6111.912931696606

In [ ]:
# Listando as métricas de validação de modelo
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [ ]:
# Executando a validação do modelo Decision Tree para os nossos dados
from sklearn.model_selection import cross_validate

modelo_arvore = DecisionTreeRegressor(max_depth = 3)

# Separa os dados em 5 grupos de Treino e Teste e gera um score para cada grupo
resultados = cross_validate(modelo_arvore, x, y, cv=10, scoring='neg_mean_squared_error')
mean_squared_errors = (resultados["test_score"]*-1)

# Informações sobre os scores, convertendo para "mean squared error"
print(f'Média: {mean_squared_errors.mean()}')
print(f'Desvio padrão: {mean_squared_errors.std()}')

Média: 6068.316591338464
Desvio padrão: 263.0416699975136


In [ ]:
from sklearn.model_selection import KFold

# Embaralhamento dos dados para evitar viés de ordenação na formação dos grupos
splitting_strategy = KFold(n_splits = 10, shuffle=True)

modelo_arvore = DecisionTreeRegressor(max_depth = 3)
resultados = cross_validate(modelo_arvore, x, y, cv=splitting_strategy, scoring='neg_mean_squared_error')
mean_squared_errors = (resultados["test_score"]*-1)

print(f'Média: {mean_squared_errors.mean()}')
print(f'Desvio padrão: {mean_squared_errors.std()}')

Média: 6057.361823141282
Desvio padrão: 98.93231689367828


In [ ]:
import numpy as np

def regressor_arvore(depth=3):
  np.random.seed(12345678)
  splitting_strategy = KFold(n_splits = 10, shuffle=True)
  modelo_arvore = DecisionTreeRegressor(max_depth = depth)
  resultados = cross_validate(modelo_arvore, x, y, cv=splitting_strategy, scoring='neg_mean_squared_error', return_train_score=True)
  print(f'Treino: {(resultados["train_score"]*-1).mean()}  | Teste: {(resultados["test_score"]*-1).mean()}')

# A partir de uma dada profundidade da árvore, o modelo fica muito tendencioso para os 
# dados de Treino, incorrendo em maior erro na predição dos dados de Teste
for i in range(1,20):
  regressor_arvore(i)

Treino: 7850.060623712294  | Teste: 7865.0044563078045
Treino: 6533.24520802353  | Teste: 6561.1343803952595
Treino: 6026.761392794219  | Teste: 6058.506173748952
Treino: 5765.883870826261  | Teste: 5821.8850793378515
Treino: 5602.271192533086  | Teste: 5685.778168141817
Treino: 5470.174137277867  | Teste: 5585.442545602948
Treino: 5369.12895877315  | Teste: 5528.579257286031
Treino: 5275.9832691820875  | Teste: 5531.826584513519
Treino: 5166.030186907752  | Teste: 5603.899682615754
Treino: 5023.928443414469  | Teste: 5733.809663995886
Treino: 4837.554028600104  | Teste: 5936.618823619619
Treino: 4603.789381074799  | Teste: 6178.227614386802
Treino: 4322.877113755471  | Teste: 6478.7672261447815
Treino: 4007.9310544414125  | Teste: 6830.638865283536
Treino: 3663.5197173494394  | Teste: 7181.435971901973
Treino: 3303.7872537688463  | Teste: 7594.981578722097
Treino: 2943.474600298102  | Teste: 8001.066744089624
Treino: 2587.6981493316553  | Teste: 8400.529986844027
Treino: 2248.34736912